In [2]:
import pandas as pd
import sqlite3
import warnings
import pyodbc
import numpy as np
import os

warnings.simplefilter('ignore')
print(pyodbc.drivers())

['ODBC Driver 18 for SQL Server', 'ODBC Driver 17 for SQL Server']


<h3> verbinding maken met SourceDataModel, en inlezen tabellen

In [3]:
# Verbindingsgegevens
server = '127.0.0.1'        
port = '1433'               
database = 'SDMProject'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Connection string
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
    "Timeout=30;"
)

# Maak verbinding met de database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Haal alle tabellen op
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = [row.TABLE_NAME for row in cursor.fetchall()]

# Dictionary om alle dataframes op te slaan
sdm_dfs = {}

# Loop door alle tabellen en laad ze in Pandas DataFrames
for table in tables:
    query = f"SELECT * FROM [{table}]"
    df = pd.read_sql(query, conn)
    sdm_dfs[table] = df
    print(f"Tabel '{table}' ingelezen met {df.shape[0]} rijen en {df.shape[1]} kolommen.")

# Sluit de verbinding
conn.close()

# Print de kolomnamen en de eerste paar rijen van elke DataFrame
for table_name, df in sdm_dfs.items():
    print(f"\nTable: {table_name}")
    print("Columns:", df.columns.tolist())
    print(df.head())
    




Tabel 'HumanResources_Department' ingelezen met 21 rijen en 5 kolommen.
Tabel 'Person_Person' ingelezen met 20056 rijen en 12 kolommen.
Tabel 'HumanResources_Employee' ingelezen met 20056 rijen en 10 kolommen.
Tabel 'Person_Address' ingelezen met 19614 rijen en 7 kolommen.
Tabel 'Production_ProductCategory' ingelezen met 12 rijen en 7 kolommen.
Tabel 'Shippers' ingelezen met 3 rijen en 3 kolommen.
Tabel 'Suppliers' ingelezen met 29 rijen en 12 kolommen.
Tabel 'Production_Product' ingelezen met 591 rijen en 34 kolommen.
Tabel 'Purchasing_Vendor' ingelezen met 104 rijen en 8 kolommen.
Tabel 'Purchasing_PurchaseOrderHeader' ingelezen met 4012 rijen en 13 kolommen.
Tabel 'Purchasing_PurchaseOrderDetail' ingelezen met 8845 rijen en 12 kolommen.
Tabel 'Sales_SalesTerritory' ingelezen met 10 rijen en 11 kolommen.
Tabel 'Sales_Store' ingelezen met 701 rijen en 5 kolommen.
Tabel 'Sales_Customer' ingelezen met 20037 rijen en 10 kolommen.
Tabel 'Sales_SalesOrderHeader' ingelezen met 32945 rijen e

<h3> dictionary maken voor goede dataframes

In [4]:
DataWarehouse_dict = {
}

<h3> HumanRescources_department

In [5]:
df_HumanRescources_Department = sdm_dfs['HumanResources_Department'].copy()

df_HumanRescources_Department.drop(columns=['ModifiedDate'], inplace=True)

DataWarehouse_dict['HumanResources_Department'] = df_HumanRescources_Department

df_HumanRescources_Department.head()

,DepartmentID,Name,GroupName,Source
0,1,Engineering,Research and Development,AdventureWorks
1,2,Tool Design,Research and Development,AdventureWorks
2,3,Sales,Sales and Marketing,AdventureWorks
3,4,Marketing,Sales and Marketing,AdventureWorks
4,5,Purchasing,Inventory Management,AdventureWorks


<h3> Person_Person

In [9]:
# Kopieer de SDM-tabel
df_Person_Person = sdm_dfs['Person_Person'].copy()

# Drop eerst de originele BusinessEntityID
df_Person_Person = df_Person_Person.drop(columns=["BusinessEntityID"])

# Hernoem MergedBusinessEntityID naar BusinessEntityID
df_Person_Person = df_Person_Person.rename(columns={
    "MergedBusinessEntityID": "BusinessEntityID"
})

# Selecteer alleen de kolommen die je nodig hebt
df_dwh_person = df_Person_Person[["BusinessEntityID", "Title", "LastName", "Source"]].copy()

# Truncate strings naar max lengte
df_dwh_person["Title"] = df_dwh_person["Title"].astype(str).str.slice(0, 50)
df_dwh_person["LastName"] = df_dwh_person["LastName"].astype(str).str.slice(0, 255)
df_dwh_person["Source"] = df_dwh_person["Source"].astype(str).str.slice(0, 50)

DataWarehouse_dict['Person_Person'] = df_dwh_person

# Preview
print(df_dwh_person.head())



   BusinessEntityID Title    LastName          Source
0                 1  None    SÃ¡nchez  AdventureWorks
1                 2  None       Duffy  AdventureWorks
2                 3  None  Tamburello  AdventureWorks
3                 4  None     Walters  AdventureWorks
4                 5   Ms.    Erickson  AdventureWorks


<h3> HumanResources_Employee

In [10]:
# Kopieer de SDM-tabel
df_HumanResources_Employee = sdm_dfs['HumanResources_Employee'].copy()

# Drop eerst de originele BusinessEntityID
df_HumanResources_Employee = df_HumanResources_Employee.drop(columns=["BusinessEntityID"])

# Hernoem MergedBusinessEntityID naar BusinessEntityID
df_HumanResources_Employee = df_HumanResources_Employee.rename(columns={
    "MergedBusinessEntityID": "BusinessEntityID"
})

# Selecteer alleen de kolommen die je nodig hebt
df_dwh_employee = df_HumanResources_Employee[["BusinessEntityID", "JobTitle", "Salary", "DepartmentID", "ManagerID", "Source"]].copy()

# Truncate strings naar max lengte
df_dwh_employee["JobTitle"] = df_dwh_employee["JobTitle"].astype(str).str.slice(0, 100)
df_dwh_employee["Source"] = df_dwh_employee["Source"].astype(str).str.slice(0, 50)

# Opslaan in het DataWarehouse
DataWarehouse_dict['HumanResources_Employee'] = df_dwh_employee

# Preview
print(df_dwh_employee.head())

   BusinessEntityID                       JobTitle  Salary  DepartmentID  \
0                 1        Chief Executive Officer     0.0          16.0   
1                 2  Vice President of Engineering     0.0           1.0   
2                 3            Engineering Manager     0.0           1.0   
3                 4           Senior Tool Designer     0.0           2.0   
4                 5                Design Engineer     0.0           1.0   

   ManagerID          Source  
0        NaN  AdventureWorks  
1        NaN  AdventureWorks  
2        NaN  AdventureWorks  
3        NaN  AdventureWorks  
4        NaN  AdventureWorks  


<h3> Person_Address

In [12]:
# Kopieer de SDM-tabel
df_Person_Address = sdm_dfs['Person_Address'].copy()

# Selecteer alleen de kolommen die je nodig hebt
df_dwh_address = df_Person_Address[["AddressID", "City", "BusinessEntityID"]].copy()

# Truncate strings naar maximale lengte
df_dwh_address["City"] = df_dwh_address["City"].astype(str).str.slice(0, 100)

# Opslaan in het DataWarehouse
DataWarehouse_dict['Person_Address'] = df_dwh_address

# Preview
print(df_dwh_address.head())

   AddressID     City  BusinessEntityID
0          1  Bothell                12
1          2  Bothell               123
2          3  Bothell               285
3          4  Bothell               251
4          5  Bothell               124


<h3> Production_ProductCategory

In [13]:
# Kopieer de SDM-tabel
df_Production_ProductCategory = sdm_dfs['Production_ProductCategory'].copy()

# Drop eerst de originele ProductCategoryID
df_Production_ProductCategory = df_Production_ProductCategory.drop(columns=["ProductCategoryID"])

# Hernoem MergedCategoryID naar ProductCategoryID
df_Production_ProductCategory = df_Production_ProductCategory.rename(columns={
    "MergedCategoryID": "ProductCategoryID"
})

# Selecteer alleen de kolommen die je nodig hebt
df_dwh_product_category = df_Production_ProductCategory[["ProductCategoryID", "Name", "Source"]].copy()

# Truncate strings naar maximale lengte
df_dwh_product_category["Name"] = df_dwh_product_category["Name"].astype(str).str.slice(0, 100)
df_dwh_product_category["Source"] = df_dwh_product_category["Source"].astype(str).str.slice(0, 50)

# Opslaan in het DataWarehouse
DataWarehouse_dict['Production_ProductCategory'] = df_dwh_product_category

# Preview
print(df_dwh_product_category.head())

   ProductCategoryID         Name          Source
0                  1        Bikes  AdventureWorks
1                  2   Components  AdventureWorks
2                  3     Clothing  AdventureWorks
3                  4  Accessories  AdventureWorks
4                  5    Beverages       NorthWind


<h3> Shippers

In [15]:
# Kopieer de SDM-tabel
df_Shippers = sdm_dfs['Shippers'].copy()

# Selecteer alleen de kolommen die je nodig hebt
df_dwh_shippers = df_Shippers[["ShipperID", "CompanyName"]].copy()

# Truncate strings naar maximale lengte
df_dwh_shippers["CompanyName"] = df_dwh_shippers["CompanyName"].astype(str).str.slice(0, 100)

# Opslaan in het DataWarehouse
DataWarehouse_dict['Shippers'] = df_dwh_shippers

# Preview
print(df_dwh_shippers.head())

   ShipperID       CompanyName
0          1    Speedy Express
1          2    United Package
2          3  Federal Shipping


<h3> Suppliers

In [16]:
# Kopieer de SDM-tabel
df_Suppliers = sdm_dfs['Suppliers'].copy()

# Selecteer alleen de kolommen die je nodig hebt
df_dwh_suppliers = df_Suppliers[["SupplierID", "CompanyName", "City", "Country", "Region"]].copy()

# Truncate strings naar maximale lengte
df_dwh_suppliers["CompanyName"] = df_dwh_suppliers["CompanyName"].astype(str).str.slice(0, 100)
df_dwh_suppliers["City"] = df_dwh_suppliers["City"].astype(str).str.slice(0, 100)
df_dwh_suppliers["Country"] = df_dwh_suppliers["Country"].astype(str).str.slice(0, 100)
df_dwh_suppliers["Region"] = df_dwh_suppliers["Region"].astype(str).str.slice(0, 100)

# Opslaan in het DataWarehouse
DataWarehouse_dict['Suppliers'] = df_dwh_suppliers

# Preview
print(df_dwh_suppliers.head())

   SupplierID                         CompanyName         City Country  \
0           1                      Exotic Liquids       London      UK   
1           2          New Orleans Cajun Delights  New Orleans     USA   
2           3           Grandma Kelly's Homestead    Ann Arbor     USA   
3           4                       Tokyo Traders        Tokyo   Japan   
4           5  Cooperativa de Quesos 'Las Cabras'       Oviedo   Spain   

     Region  
0      None  
1        LA  
2        MI  
3      None  
4  Asturias  


<h3>Production_Product

In [ ]:
CREATE TABLE Production_Product (
    ProductID INT,
    Name VARCHAR(100),
    StandardCost DECIMAL(10,2),
    ProductCategoryID INT,
    QuantityPerUnit VARCHAR(50),
    DaysToManufacture INT,
    ProductLine VARCHAR(30),
    UnitPrice DECIMAL(10,2),
    UnitsInStock INT,
    UnitsOnOrder INT,
    SellStartDate DATE,
    SellEndDate DATE,
    DiscontiuedDate DATE,
    SupplierID INT,
    FOREIGN KEY (SupplierID) REFERENCES Suppliers(SupplierID),
    FOREIGN KEY (ProductCategoryID) REFERENCES Production_ProductCategory(ProductCategoryID),
    FOREIGN KEY (SupplierID) REFERENCES Purchasing_Vendor(BusinessEntityID)
);